# 🧠 Model Definition Notebook
Defines the multimodal CNN + BiLSTM + Static fusion network with Gradient Reversal Layer.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self, hp_lambda=1.0):
        super().__init__()
        self.hp_lambda = hp_lambda
    def call(self, x):
        @tf.custom_gradient
        def _reverse(x):
            def grad(dy): return -self.hp_lambda * dy
            return x, grad
        return _reverse(x)
def build_fusion_model():
    audio_in = layers.Input((128,128,1)); phys_in = layers.Input((300,10)); static_in = layers.Input((8,))
    a = layers.Conv2D(32,3,activation='relu')(audio_in); a = layers.MaxPool2D()(a); a = layers.Flatten()(a)
    p = layers.Bidirectional(layers.LSTM(64))(phys_in)
    s = layers.Dense(32,activation='relu')(static_in)
    concat = layers.Concatenate()([a,p,s])
    x = layers.Dense(128,activation='relu')(concat)
    class_out = layers.Dense(2,activation='softmax',name='class_out')(x)
    grl = GradientReversalLayer(1.0)(concat)
    d = layers.Dense(64,activation='relu')(grl); domain_out = layers.Dense(3,activation='softmax',name='domain_out')(d)
    return Model([audio_in,phys_in,static_in],[class_out,domain_out])
model = build_fusion_model()
model.summary()